<a href="https://colab.research.google.com/github/RocioLiu/ML_Resources/blob/master/Geron_ch12_Custom_Models_and_Training_with_TensorFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Using TensorFlow like Numpy**

In [1]:
%tensorflow_version 2.x

TensorFlow 2.x selected.


In [0]:
import tensorflow as tf
from tensorflow import keras

In [3]:
print(tf.__version__)
print(keras.__version__)

2.1.0-rc1
2.2.4-tf


### **Tensors and Operations**

We can create a tensor with `tf.constant()` .

In [0]:
tf.constant([[1., 2., 3.], [4., 5., 6.]]) # matrix

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [0]:
tf.constant(42) # scalar

<tf.Tensor: shape=(), dtype=int32, numpy=42>

Just like an `ndarray`, a `tf.Tensor` has a shape and a data type (`dtype`)

In [0]:
t = tf.constant([[1., 2., 3.], [4., 5., 6.]])
t.shape

TensorShape([2, 3])

In [0]:
t.dtype

tf.float32

Indexing works much like in Numpy:

In [0]:
t[:, 1:]

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[2., 3.],
       [5., 6.]], dtype=float32)>

In [0]:
t[..., 1, tf.newaxis]

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[2.],
       [5.]], dtype=float32)>

In [0]:
t[..., 1]

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([2., 5.], dtype=float32)>

Most importantly, all sorts of tensor operations are available:

In [0]:
t + 10

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[11., 12., 13.],
       [14., 15., 16.]], dtype=float32)>

In [0]:
tf.square(t)

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[ 1.,  4.,  9.],
       [16., 25., 36.]], dtype=float32)>

In [0]:
t @ tf.transpose(t) # The @ operator is for matrix multiplication: it's equivalent to calling the tf.matmul() function.

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[14., 32.],
       [32., 77.]], dtype=float32)>

### **Tensors and Numpy**
Tensors play nice with Numpy: we can create a tensor from a Numpy array, and even apply TensorFlow operations to Numpy arrays and Numpy operations to tensors.

In [0]:
import numpy as np
a = np.array([2., 4., 5.])
tf.constant(a)


<tf.Tensor: shape=(3,), dtype=float64, numpy=array([2., 4., 5.])>

In [0]:
t.numpy() # or np.array(t)

array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)

In [0]:
tf.square(a)

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([ 4., 16., 25.])>

In [0]:
np.square(t)

array([[ 1.,  4.,  9.],
       [16., 25., 36.]], dtype=float32)

### **Type Conversion**
TensorFlow does not perfrom any type conversions automatically: it just raises an exception if we try to execute an operation on tensors with incompatible types.

In [0]:
tf.constant(2.) + tf.constant(40)

InvalidArgumentError: ignored

In [0]:
tf.constant(2.) + tf.constant(40., dtype=tf.float64)

InvalidArgumentError: ignored

In [0]:
t2 = tf.constant(40., dtype=tf.float64)
tf.constant(2.0) + tf.cast(t2, tf.float32)

<tf.Tensor: shape=(), dtype=float32, numpy=42.0>

### **Variables**
The `tf.Tensor` values we've seen so far are immutable. But if we want it to change over time, what we need is a `tf.Variable` :

In [0]:
v = tf.Variable([[1., 2., 3.], [4., 5., 6.]])
v

<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

The `tf.Variable` can be modified in place using the `assign()` method.

In [0]:
v.assign(2 * v)

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2.,  4.,  6.],
       [ 8., 10., 12.]], dtype=float32)>

In [0]:
v[0, 1].assign(42)

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2., 42.,  6.],
       [ 8., 10., 12.]], dtype=float32)>

In [0]:
v[:, 2].assign([0., 1.])

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2., 42.,  0.],
       [ 8., 10.,  1.]], dtype=float32)>

In [0]:
v.scatter_nd_update(indices=[[0,0], [1,2]], updates=[100.,200.])

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[100.,  42.,   0.],
       [  8.,  10., 200.]], dtype=float32)>

## **Customizing Models and Training Algorithm**

### **Custom Loss Functions**
Create a Huber loss myself that takes the lables and predictions as arguments, and use TensorFlow operations to compute every instance's loss

In [0]:
def huber_fn(y_true, y_pred):
  error = y_true - y_pred
  is_small_error = tf.abs(error) < 1
  squared_loss = tf.square(error) / 2
  linear_loss = tf.abs(error) - 0.5
  return tf.where(is_small_error, squared_loss, linear_loss)

Now we can use this loss when we compile the Keras model, the train our model:

In [0]:
model.compile(loss=huber_fn, optimizer="nadam")
model.fit(X_train, y_train, [...])

### **Saving and Loading Models That Contain Custom Components**
When we load a model containing custom objects, we need to provide a dictionary that maps the function name to the actual function.


In [0]:
model = keras.models.load_model("my_model_with_a_custom_loss.h5",
                                custom_objects={"huber_fn":huber_fn})

If we want to create a different threshold of loss, One solution is to create a function that creates a configured loss function

In [0]:
def create_huber(threshold=1.0):
  def huber_fn(y_true, y_pred):
    error = y_true - y_pred
    is_small_error = tf.abs(error) < threshold
    squared_loss = tf.square(error) / 2
    linear_loss = threshold * tf.abs(error) - threshold**2 / 2
    return tf.where(is_small_error, squared_loss, linear_loss)
  return huber_fn

model.compile(loss=create_huber(2.0), optimizer="nadam")

Unfortunately, whan we save the model, the `threshold` will not be saved. We need to specify the `threshold` value when loading the model (note that the name to use is `"huber_fn"`, which is the function we gave Keras, not the name of the function that created it)

In [0]:
model = keras.models.load_model("my_model_with_a custom_loss_threshold_2.h5",
                                custom_objects={"huber_fn": create_huber(2.0)})

We can solve this by creating a subclass of the `keras.losses.Loss` class, and then implementing its `get_config()` method:

In [0]:
class HuberLoss(keras.losses.Loss):
  def __init__(self, threshold=1.0, **kwargs):
    self.threshold = threshold
    super().__init__(**kwargs)
  def call(self, y_true, y_pred):
    error = y_true - y_pred
    is_small_error = tf.abs(error) < self.threshold
    squared_loss = tf.square(error) / 2
    linear_loss = self.threshold * tf.abs(error) - threshold**2 / 2
    return tf.where(is_small_error, squared_loss, linear_loss)
  def get_config(self):
    base_config = super().get_config()
    return {**base_config, "threshold": self.threshold}

In [0]:
model.compile(loss=HuberLoss(2.), optimizer='nadam')

When we save the model, the threshold will be saved along with it; and when we load the model. we just need to map the class name to the class itself:

In [0]:
model = keras.models.load_model("my_model_with_a_custom_loss_class.h5",
                                custom_objects={"HuberLoss": HuberLoss})